---

## EVCS POI Candidate Selection

This section outlines the process of selecting POI (Point of Interest) candidates categorized by region.

- The above approach represents the process of selecting POI candidates categorized by region. POI filtering was applied when selecting POI candidates for all regions.
  
  - Before filtering, OSM_POI data was imported, and POIs were extracted for each categorized region (Atlanta, Suburban, Rural) using GIS tools.
  
  - For each extracted POI candidate, filtering was performed using the 'fclass' column, which contains information about building usage, utilizing GIS tools.

<br>

---

### Summary

- **DCFC POI Candidate Selection**: Georgia, using GIS tools & filtering.  
<br>
---


In [ ]:
import geopandas as gpd
import pandas as pd

In [ ]:
gpkg_file_path = "Path To your Suburban Region POI"
output_path =  "suburban_poi_candidate.gpkg"
region = 'suburban'

def process_poi_data(region, gpkg_file_path, output_path):
    # Define file paths and fclass categories based on the region
    if region == 'suburban':
        selected_fclasses = ['school', 'library', 'bank', 'fast_food', 'restaurant', 'museum', 'college', 
                             'hospital', 'university', 'town_hall', 'stadium', 'courthouse', 'public_building', 
                             'hotel', 'pharmacy', 'supermarket', 'department_store', 'gift_shop', 'car_rental', 
                             'zoo', 'motel', 'mall', 'camp_site', 'general']  # Suburban-specific categories
    # Read the GPKG file
    poi_gdf = gpd.read_file(gpkg_file_path)

    # Filter the POI data based on the selected fclass categories
    selected_poi_gdf = poi_gdf[poi_gdf['fclass'].isin(selected_fclasses)]
    
    # Get the count of filtered features
    selected_feature_count = selected_poi_gdf.shape[0]
    print(f"Selected feature count for {region}: {selected_feature_count}")

    # Save the filtered data to a new GPKG file
    selected_poi_gdf.to_file(output_path, driver='GPKG')
    print(f"Filtered POI data saved for {region} at {output_path}")

# Example usage:
process_poi_data(region, gpkg_file_path, output_path ) 

## Case 4: Suburban DCFC EVCS Charger Initial Point Selection

In **Suburban areas**, high-speed chargers (DCFC) tend to be installed at intersections of major highways. To reflect this trend, the selection process considered the **road network**, **demand map**, and **POI** data. The DCFC EVCS locations in suburban areas are generally clearer compared to urban settings, and because the number of POI candidates is relatively low, running a full optimization was deemed unnecessary. Therefore, the following algorithm was used to install DCFC EVCS in all suburban areas.

### Algorithm:

1. **Selecting POIs near Motorway and Trunkway Intersections**:
   - Using **Motorway** and **Trunkway** road network data, the algorithm identifies intersections.
   - For each intersection, POIs within a **2410-meter radius** are searched.
   - Among the selected POIs, those belonging to **priority categories** (e.g., `department_store`, `hospital`, `market_place`, etc.) are prioritized.
   - If no POI is found within the priority categories, the nearest POI is randomly selected, and a **DCFC EVCS** is installed.

2. **Selecting POIs based on GeoTIFF Demand Data for Unselected Suburban Areas**:
   - For areas not covered by the intersection-based selection, the **GeoTIFF demand map** is used to ensure POIs are placed in areas with higher demand.
   - Using the `rasterio` library, the pixel value for each POI location is extracted from the demand map.
   - The algorithm first checks if a POI near a high-demand area belongs to a priority category, and if no such POI is found, it selects the nearest POI based on demand values to install the **DCFC EVCS**.

3. **Selecting POIs within Urban Polygons**:
   - The number of POIs to be selected in each **urban polygon** is determined by the variable **‘p’** associated with each polygon.
   - For each urban polygon, POIs are selected randomly, prioritizing those in **priority categories**. If the selected number of POIs is insufficient, additional POIs are selected based on **GeoTIFF demand data**.
   - If the polygon contains fewer POIs than required, additional POIs are chosen near high-demand pixels from the demand map, and the **DCFC EVCS** is installed.

By following the above steps, **DCFC EVCS** installations were successfully allocated across all suburban areas.


In [ ]:
import geopandas as gpd
import rasterio
import pandas as pd
import numpy as np
from shapely.geometry import Point
from shapely.ops import unary_union
from tqdm import tqdm

# File paths
urban_polygon_path = 'Path to your Suburban Polygon.gpkg'  # Urban Polygon
motorway_path = 'Path to your Motorways.gpkg'  # Motorway
trunk_path = 'Path to your  Trunk.gpkg'  # Trunk
poi_path = 'Path to your suborban_poi_candidate.gpkg'  # POI
tif_file = 'Path to your Demand map'  # GeoTIFF

# Parameters
radius_meters = 3000  # 3 km 반경
selected_category = [
    'department_store', 'bank', 'hotel', 'theatre', 'town_hall', 'hospital', 
    'market_place', 'mall', 'public_building', 'cinema', 'vending_parking', 
    'general', 'cafe', 'fast_food', 'pharmacy', 'clothes'
]

# Load data
urban_polygon_gdf = gpd.read_file(urban_polygon_path)
motorway_gdf = gpd.read_file(motorway_path)
trunk_gdf = gpd.read_file(trunk_path)
poi_gdf = gpd.read_file(poi_path)

# Ensure correct CRS (EPSG:3857)
if urban_polygon_gdf.crs.to_string() != 'EPSG:3857':
    urban_polygon_gdf = urban_polygon_gdf.to_crs('EPSG:3857')

if motorway_gdf.crs.to_string() != 'EPSG:3857':
    motorway_gdf = motorway_gdf.to_crs('EPSG:3857')

if trunk_gdf.crs.to_string() != 'EPSG:3857':
    trunk_gdf = trunk_gdf.to_crs('EPSG:3857')

if poi_gdf.crs.to_string() != 'EPSG:3857':
    poi_gdf = poi_gdf.to_crs('EPSG:3857')

# 1. Motorway와 Trunk 간의 교차점 추출
motorway_union = unary_union(motorway_gdf.geometry)
trunk_union = unary_union(trunk_gdf.geometry)

intersection_points = motorway_union.intersection(trunk_union)

# 교차점이 여러 개일 경우 MultiPoint 처리
if intersection_points.geom_type == 'MultiPoint':
    intersection_points_gdf = gpd.GeoDataFrame(geometry=list(intersection_points.geoms), crs='EPSG:3857')
else:
    intersection_points_gdf = gpd.GeoDataFrame(geometry=[intersection_points], crs='EPSG:3857')

# 2. 교차점 반경 내에서 POI 찾기
def get_pois_within_radius(intersection_gdf, poi_gdf, radius_meters):
    selected_pois = []

    for point in intersection_gdf.geometry:
        # POI와 Intersection Point 사이의 거리 계산
        poi_gdf['distance'] = poi_gdf.geometry.apply(lambda x: point.distance(x))
        
        # 반경 내에 있는 POI 추출
        pois_within_radius = poi_gdf[poi_gdf['distance'] <= radius_meters]
        
        # 선택된 POI 추가
        selected_pois.append(pois_within_radius)
    
    # 모든 POI 합치기
    return gpd.GeoDataFrame(pd.concat(selected_pois, ignore_index=True), crs=poi_gdf.crs)

# 3. GeoTIFF에서 픽셀 값을 추출해 POI에 할당
def get_nearest_tif_value(poi_gdf, tif_file):
    with rasterio.open(tif_file) as tif:
        tif_data = tif.read(1)  # 첫 번째 밴드 읽기
        transform = tif.transform

        # POI의 좌표에 대응하는 GeoTIFF 픽셀 값 할당
        poi_gdf['tif_value'] = poi_gdf.geometry.apply(lambda point: get_pixel_value(point, tif_data, transform))

    return poi_gdf

# 특정 좌표에 대해 GeoTIFF에서 픽셀 값을 반환하는 함수
def get_pixel_value(point, tif_data, transform):
    row, col = ~transform * (point.x, point.y)  # 좌표를 픽셀 좌표로 변환
    row, col = int(row), int(col)

    if 0 <= row < tif_data.shape[0] and 0 <= col < tif_data.shape[1]:
        return tif_data[row, col]
    else:
        return np.nan  # 범위 밖일 경우 NaN 반환

# GeoTIFF 값이 높은 곳에 POI를 배치하는 함수
def assign_pois_from_tif(urban_polygon, tif_file, num_pois_needed, pois_gdf):
    with rasterio.open(tif_file) as tif:
        transform = tif.transform
        tif_data = tif.read(1)
        
        # Urban Polygon 경계 내에서 GeoTIFF 값이 높은 픽셀 선택
        bounds = urban_polygon.geometry.bounds
        x_min, y_min, x_max, y_max = bounds
        
        pixel_values = []
        for x in np.arange(x_min, x_max, tif.res[0]):
            for y in np.arange(y_min, y_max, tif.res[1]):
                point = gpd.points_from_xy([x], [y])[0]
                if urban_polygon.geometry.contains(point):
                    row, col = ~transform * (x, y)
                    row, col = int(row), int(col)
                    if 0 <= row < tif_data.shape[0] and 0 <= col < tif_data.shape[1]:
                        pixel_value = tif_data[row, col]
                        pixel_values.append((point, pixel_value))

        # GeoTIFF 값이 높은 순서대로 정렬
        pixel_values = sorted(pixel_values, key=lambda x: x[1], reverse=True)

        # 선택된 픽셀에 해당하는 POI들을 탐색
        selected_pois = []
        for point, pixel_value in pixel_values:
            pois_near_pixel = pois_gdf[pois_gdf.distance(point) < tif.res[0]]
            pois_in_category = pois_near_pixel[pois_near_pixel['category'].isin(selected_category)]
            
            if not pois_in_category.empty:
                selected_pois.append(pois_in_category.sample(n=1))  # 우선순위 POI가 있으면 선택
            elif not pois_near_pixel.empty:
                selected_pois.append(pois_near_pixel.sample(n=1))  # 우선순위 POI가 없으면 다른 POI 선택
            
            if len(selected_pois) >= num_pois_needed:
                break

        return gpd.GeoDataFrame(pd.concat(selected_pois, ignore_index=True), crs=urban_polygon.crs)

# 4. 도시 폴리곤 안에서 POI 선택
def select_random_pois(urban_gdf, pois_gdf, tif_file):
    selected_pois = []
    pois_gdf = get_nearest_tif_value(pois_gdf, tif_file)

    for fid, urban_polygon in tqdm(urban_gdf.iterrows(), total=len(urban_gdf), desc="Processing Urban Polygons"):
        pois_within_boundary = pois_gdf[pois_gdf.within(urban_polygon.geometry)]
        num_pois_to_select = int(urban_polygon['p'])

        if len(pois_within_boundary) >= num_pois_to_select:
            selected_pois.append(pois_within_boundary.sample(num_pois_to_select))
        else:
            remaining_pois_needed = num_pois_to_select - len(pois_within_boundary)
            tif_pois = assign_pois_from_tif(urban_polygon, tif_file, remaining_pois_needed, pois_gdf)
            selected_pois.append(tif_pois)

    return gpd.GeoDataFrame(pd.concat(selected_pois, ignore_index=True), crs=pois_gdf.crs)

# 교차점 반경 내에서 POI 선택
pois_near_intersections = get_pois_within_radius(intersection_points_gdf, poi_gdf, radius_meters)

# 도시 폴리곤 내에서 POI를 선택하고 GeoTIFF로 추가 배치
selected_pois = select_random_pois(urban_polygon_gdf, poi_gdf, tif_file)

# 결과 저장
selected_pois.to_file('suburban_dcfc_selection.gpkg', driver='GPKG')

print(f"Total selected POIs: {len(selected_pois)}")